# Imports

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import math
from scipy import stats
from scipy.stats import shapiro
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import mannwhitneyu

In [4]:
item_prices = pd.read_csv("item_prices.csv")
sample_a = pd.read_csv("sample_a.csv")
sample_b = pd.read_csv("sample_b.csv")
sample_c = pd.read_csv("sample_c.csv")

# 0 introduction

In [17]:
print('Total duplicates item_prices: {}'.format(item_prices.duplicated().sum().sum()))
print('Total duplicates sample_b: {}'.format(sample_b.duplicated().sum().sum()))
print('Total duplicates sample_a: {}'.format(sample_a.duplicated().sum().sum()))
print('Total duplicates sample_c: {}'.format(sample_c.duplicated().sum().sum()))
print('-'*40)
print('Total missing values item_prices: {}'.format(item_prices.isnull().sum().sum()))
print('Total missing values sample_b: {}'.format(sample_b.isnull().sum().sum()))
print('Total missing values sample_c: {}'.format(sample_c.isnull().sum().sum()))
print('Total missing values sample_a: {}'.format(sample_a.isnull().sum().sum()))
print('-'*40)  
print('Shape item_prices: {}'.format(item_prices.shape))
print('Shape sample_b: {}'.format(sample_b.shape))
print('Shape sample_c: {}'.format(sample_c.shape)) 
print('Shape sample_a: {}'.format(sample_a.shape))

Total duplicates item_prices: 0


Total duplicates sample_b: 0
Total duplicates sample_a: 0
Total duplicates sample_c: 0
----------------------------------------
Total missing values item_prices: 0
Total missing values sample_b: 0
Total missing values sample_c: 0
Total missing values sample_a: 0
----------------------------------------
Shape item_prices: (1000, 2)
Shape sample_b: (1198438, 3)
Shape sample_c: (1205510, 3)
Shape sample_a: (1188912, 3)
